In [0]:
import tensorflow as tf, pandas as pd, numpy as np, time, os

In [5]:
!pip install pydub

In [0]:
# pydub
from pydub import AudioSegment

In [9]:
!git clone https://github.com/RedbirdTaiwan/silic.git

Cloning into 'silic'...
remote: Enumerating objects: 18, done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 18
Unpacking objects: 100% (18/18), done.


In [46]:
!ls silic/
!ls silic/model
!ls silic/sample

LICENSE  model	README.md  sample
conv_labels.txt  frozen_graph.pb  sound_class.txt
25068.mp3  87864.mp3


In [0]:
class Autolabel():
  def __init__(self, label_file, graph_file):
    self.label_file = label_file
    self.graph_file = graph_file
    self.sound = ''
    self.results = {}

  def audiofile(self, audio_file):
    frame_rate = 30000
    filext = audio_file[-3:].lower()
    if filext == "mp3":
        self.sound = AudioSegment.from_mp3(audio_file)
    elif filext == "wma":
        self.sound = AudioSegment.from_file(audio_file, "wma")
    elif filext == "m4a":
        self.sound = AudioSegment.from_file(audio_file, "m4a")
    elif filext == "ogg":
        self.sound = AudioSegment.from_ogg(audio_file)
    else:
        self.sound = AudioSegment.from_wav(audio_file)
    if self.sound.frame_rate > frame_rate:
        self.sound = self.sound.set_frame_rate(frame_rate)
    if self.sound.channels > 1:
        self.sound = self.sound.split_to_mono()[0]
    if not self.sound.sample_width == 2:
        self.sound = soself.soundund.set_sample_width(2)
    self.sound = self.sound - (self.sound.dBFS+25)

  def ailabel(self, **kwargs):
    if not len(self.sound):
      print('use ".audiofile(<audio_file_path>)" to import a recording')
      return false
    if not 'step' in kwargs.keys():
      step = 3000
    else:
      step = kwargs['step']
    sample_rate = 30000
    clip_duration_ms = 3000
    labels_list = [line.rstrip() for line in tf.io.gfile.GFile(self.label_file)]
    with tf.io.gfile.GFile(self.graph_file, 'rb') as f:
      graph_def = tf.compat.v1.GraphDef()
      graph_def.ParseFromString(f.read())
      tf.import_graph_def(graph_def, name='')
    sess = tf.compat.v1.Session()
    softmax_tensor = sess.graph.get_tensor_by_name('labels_softmax:0')
    self.results = {k:[] for k in labels_list[2:]}
    for t in range(0,len(self.sound),step):
      if (t + clip_duration_ms) < len(self.sound):
        soundclip = self.sound[t:t+clip_duration_ms]
        soundclip.export('tmp.wav', format="wav")
        with open('tmp.wav', 'rb') as wav:
          wav_data2 = wav.read()
        predictions, = sess.run(softmax_tensor, {'wav_data:0': wav_data2})
        for node_id in range(2, len(labels_list)):
          song_of_bird = labels_list[node_id]
          score = round(predictions[node_id],3)
          self.results[song_of_bird].append([t,score])

In [0]:
test = Autolabel('silic/model/conv_labels.txt', 'silic/model/frozen_graph.pb')

In [0]:
test.audiofile('silic/sample/87864.mp3')

In [0]:
test.ailabel()

In [47]:
type(test.sound)

pydub.audio_segment.AudioSegment

In [44]:
results = test.results
soundclass = pd.read_csv('silic/model/sound_class.txt', index_col=0, sep='\t').T.to_dict()
for soundid, scores in zip(results.keys(), results.values()):
  for score in scores:
    if score[1] >= 0.1:
      sound = soundclass[int(soundid)]['common_name'] + ':' + soundclass[int(soundid)]['class_name']
      print(sound, int(score[0]/1000), score[1])

青背山雀:S-01 0 0.108
青背山雀:S-01 6 0.131
青背山雀:S-01 9 0.106
青背山雀:S-01 12 0.134
青背山雀:S-01 15 0.176
青背山雀:S-01 21 0.168
青背山雀:S-01 27 0.182
青背山雀:S-01 33 0.173
青背山雀:S-01 36 0.103
青背山雀:S-01 39 0.162
青背山雀:S-01 42 0.143
青背山雀:S-01 45 0.194
青背山雀:S-01 48 0.127
青背山雀:S-01 54 0.183
黃眉柳鶯:C-01 39 0.102
黃眉柳鶯:C-01 51 0.103
拉都希氏赤蛙:S-01 18 0.111
拉都希氏赤蛙:S-01 30 0.131
深山鶯:C-01 0 0.165
深山鶯:C-01 6 0.179
深山鶯:C-01 9 0.228
深山鶯:C-01 12 0.106
深山鶯:C-01 15 0.188
深山鶯:C-01 21 0.214
深山鶯:C-01 27 0.11
深山鶯:C-01 33 0.135
深山鶯:C-01 36 0.181
深山鶯:C-01 39 0.187
深山鶯:C-01 42 0.168
深山鶯:C-01 45 0.167
深山鶯:C-01 48 0.147
深山鶯:C-01 51 0.135
深山鶯:C-01 54 0.151
